In [1]:
import pandas as pd
import quantaxis_ext
from QUANTAXIS import QA_fetch_stock_day_adv
from QUANTAXIS import QA_fetch_index_day_adv
import ploter
import talib
import numpy as np
import settings
import units
import header
import calculator as calc
import matplotlib.pyplot as plt

#设定绘图的默认大小
import matplotlib
matplotlib.rcParams["figure.figsize"]=[16,5]

#加载 seaborn，并且设置默认使用 seaborn
import seaborn as sns
sns.set()

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Noto Sans CJK SC','SimHei']
matplotlib.rcParams['axes.unicode_minus']=False #用来正常显示负号

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

SyntaxError: invalid syntax (QAQuery_Async.py, line 64)

In [ ]:
START_VALS = 1000000 # 初始投资总额
START = '2018-01-01' # 初始投资日期
END = '2018-12-31' # 结束投资日期
# 投资组合：贵州茅台、中国平安、招商银行、格力电器
SYMBOLS = ['600519','601318','600036','000651'] 
# 比率分配
ALLOCS = [0.4,0.4,0.1,0.1]
ZS_CODE='000300'#指数代码。沪深300

## 日回报率

In [ ]:
daily_return=quantaxis_ext.fetch_index_stock_daily_adv(SYMBOLS, ZS_CODE, START, END).pct_change()[1:]
daily_return.head()

## 组合的日回报率

In [ ]:
symbol_daily_return=daily_return.drop(columns=['zs_'+ZS_CODE])
symbol_daily_return.head()

## 组合日收益协方差矩阵

In [ ]:
symbol_cov_mat=symbol_daily_return.cov()
symbol_cov_mat.head()

## 组合日收益均值

In [ ]:
symbol_avg_rets =symbol_daily_return.mean()
symbol_avg_rets.head()

## 按照默认的组合分配比率，默认的初始资金，计算期末时的价值

In [ ]:
port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,ALLOCS,START_VALS)

In [ ]:
print('按照最默认组合 {0}\n计算后的期末余额为 {1}。\n比期初降低 {2:.2%}。'.format(ALLOCS,
                                                        port_vals.iloc[-1].sum(),
                                                        port_vals.iloc[-1].sum()/START_VALS-1))

## 计算最小方差组合（长期投资，比率不可为负）

In [ ]:
import portfolioopt as opt

In [ ]:
weight_min_portfolio=opt.min_var_portfolio(symbol_cov_mat)
weight_min_portfolio

In [ ]:
min_portfolio_port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,weight_min_portfolio.values,START_VALS)

In [ ]:
print('按照最小方差组合（长期投资，比率不可为负） {0}\n计算后的期末余额为 {1}\n比默认组合增加 {2:.2%}。'.format(weight_min_portfolio.values,
                                                        min_portfolio_port_vals.iloc[-1].sum(),
                                                        min_portfolio_port_vals.iloc[-1].sum()/port_vals.iloc[-1].sum()-1))

## 计算最小方差组合（短期投资，比率可为负）

In [ ]:
weight_min_portfolio_short=opt.min_var_portfolio(symbol_cov_mat,allow_short=True)
weight_min_portfolio_short

In [ ]:
min_portfolio_short_port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,weight_min_portfolio_short.values,START_VALS)

In [ ]:
print('按照最小方差组合（短期投资，比率可为负） {0}\n计算后的期末余额为 {1}\n比默认组合增加 {2:.2%}。'.format(weight_min_portfolio_short.values,
                                                        min_portfolio_short_port_vals.iloc[-1].sum(),
                                                        min_portfolio_short_port_vals.iloc[-1].sum()/port_vals.iloc[-1].sum()-1))

## 计算Markowitz组合

In [ ]:
weight_markowitz=opt.markowitz_portfolio(symbol_cov_mat,symbol_avg_rets,symbol_avg_rets.quantile(0.7))
weight_markowitz

In [ ]:
markowitz_port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,weight_markowitz.values,START_VALS)

In [ ]:
print('按照Markowitz组合 {0}\n计算后的期末余额为 {1}\n比默认组合增加 {2:.2%}。'.format(weight_markowitz.values,
                                                        markowitz_port_vals.iloc[-1].sum(),
                                                        markowitz_port_vals.iloc[-1].sum()/port_vals.iloc[-1].sum()-1))

## 计算“切线资产组合” (tangency portfolio)

In [ ]:
weight_tangency=opt.tangency_portfolio(symbol_cov_mat, symbol_avg_rets)
weight_tangency

In [ ]:
tangency_port_vals=calc.calc_residual_value_of_portfolio(SYMBOLS,ZS_CODE,START,END,weight_tangency.values,START_VALS)

In [ ]:
print('按照切线资产组合 {0}\n计算后的期末余额为 {1}\n比默认组合增加 {2:.2%}。'.format(weight_tangency.values,
                                                        tangency_port_vals.iloc[-1].sum(),
                                                        tangency_port_vals.iloc[-1].sum()/port_vals.iloc[-1].sum()-1))